지역
- 파주시
- A군, 수요 데이터 없음

중심점
- 행복주택

고려한 요소들
- 공영주차장
- 쏘카존
- 지하철
- 아파트

스케일링 여부
- 평균으로 나눔

In [1]:
import pandas as pd
import numpy as np
import dataframe_image as dfi

In [2]:
# 데이터프레임을 그림 파일로 저장
save_option = 3 # 1 : dfi / 2 : df2img / 3 : do nothing
import df2img
def save(df, path):
    if save_option == 1:
        dfi.export(df, path)
    if save_option == 2:
        df2img.save_dataframe(fig=df2img.plot_dataframe(df), filename=path)
    if save_option == 3:
        pass

각 군집별 도시들
- cluster A : ['고양시 덕양구', '김포시', '성남시 분당구', '시흥시', '용인시 기흥구', '의정부시', '파주시']
- cluster B : ['구리시', '군포시', '성남시 수정구', '성남시 중원구', '수원시 장안구', '수원시 팔달구', '안성시', '안양시 만안구', '양주시', '오산시', '용인시 처인구', '의왕시', '이천시']
- cluster C : ['남양주시', '부천시', '화성시']
- cluster D : ['가평군', '과천시', '동두천시', '양평군', '여주시', '연천군', '포천시']
- cluster E : ['고양시 일산동구', '고양시 일산서구', '광명시', '광주시', '수원시 권선구', '수원시 영통구', '안산시 단원구', '안산시 상록구', '안양시 동안구', '용인시 수지구', '하남시']
- cluster F : ['평택시']

각 군집별 (수요 데이터가 있는) 도시들
- Cluster A : ['고양시 덕양구', '김포시', '성남시 분당구', '의정부시']
- Cluster B : ['구리시', '성남시 수정구', '성남시 중원구', '안양시 만안구']
- Cluster C : ['남양주시', '화성시']
- Cluster D : ['양평군']
- Cluster E : ['고양시 일산동구', '고양시 일산서구', '광명시', '안양시 동안구', '하남시']
- Cluster F : []

파주시는 A군에 속해있고, 수요데이터는 없는 도시이다.

# 데이터 작업

## 행복주택

In [35]:
happys = pd.read_csv("../../data/data_b_raw/2b206_0111_경기도 행복주택주거지현황.csv")
happys.rename(columns={'주소':'행복주택 주소'}, inplace=True)
happys = happys[happys['지역']=='파주시'].reset_index(drop=True)
happy_names = happys['행복주택 주소']
happys = happys[['경도', '위도']].rename(columns={'경도':'x', '위도':'y'})
happys = happys.values
happys

array([[126.76816479,  37.73036766],
       [126.72139206,  37.70817011],
       [126.8776337 ,  37.84718409],
       [126.69232552,  37.73169423],
       [126.70674563,  37.72692207],
       [126.70825863,  37.71424918],
       [126.7333745 ,  37.73314793],
       [126.71982608,  37.70843444]])

## 아파트

In [4]:
apartments = pd.read_csv("../../data/data_b_raw/2b212_0205_경기도 파주시_공동주택현황_20221109.csv")
apartments = apartments[1:217]
apartments.rename(columns={'도로명주소':'아파트 주소', '단지명':'아파트 이름'}, inplace=True)
apartment_names = apartments[['아파트 이름', '아파트 주소']]
apartments = apartments[['Longitude','Latitude']]
apartments.rename(columns={'Longitude':'x', 'Latitude':'y'}, inplace=True)
apartments = apartments.astype(float).values
apartments.shape

(216, 2)

## 공영주차장

In [5]:
parks = pd.read_csv("../../data/data_b_raw/2b208_0202_한국교통안전공단_전국공영주차장정보_20191224.csv", encoding='CP949')
parks.rename(columns={'주차장지번주소':'주차장 주소'}, inplace=True)
parks = parks[parks['지역구분']=='경기도'].reset_index(drop=True)
parks = parks[parks['지역구분_sub']=='파주시'].reset_index(drop=True)
park_names = parks[['주차장 주소']]
parks = parks[['경도', '위도']].rename(columns={'경도':'x','위도':'y'})
parks = parks.values
parks.shape

(42, 2)

## 쏘카존

In [6]:
# 쏘카존 갯수 데이터(김선중)
# 출처 : https://d3s0tskafalll9.cloudfront.net/media/documents/20210823_socar_zone.csv (going deeper 4-6)
zones = pd.read_csv("../../data/data_b_raw/2b201_1228_20210823_socar_zone.csv")
zones.rename(columns={'zone_name':'존 이름'}, inplace=True)
zones = zones[zones['region2']=='파주시'].reset_index(drop=True)
zone_names = zones[['존 이름']]
zones = zones[['lng','lat']]
zones.rename(columns={'lng':'x','lat':'y'}, inplace=True)
zones = zones.values
zones.shape

(36, 2)

## 지하철역

In [7]:
stations = ['야당', '운정', '금릉', '금촌', '월롱', '파주', '문산', '운천', '임진강']
subways = pd.read_csv("../../data/data_b_raw/2b213_0205_국가철도공단_경의중앙선_역위치_20221122.csv", encoding='CP949')
subways = subways[subways['역명'].isin(stations)]
subways.rename(columns={'역명':'역 이름'}, inplace=True)
subways = subways[['역 이름', '경도', '위도']].reset_index(drop=True)
subways.loc[4] =['운천', 126.769769, 37.880141]
subways.loc[7] =['파주', 126.792437, 37.815670]
subways.loc[8] =['월롱', 126.792623, 37.796244]
subway_names = subways[['역 이름']]
subways = subways[['경도', '위도']].rename(columns={'위도':'y','경도':'x'})
subways = subways[['x', 'y']].values
subways

array([[126.761535,  37.712749],
       [126.767076,  37.725441],
       [126.774515,  37.766106],
       [126.787531,  37.854565],
       [126.769769,  37.880141],
       [126.765401,  37.751295],
       [126.746777,  37.888364],
       [126.792437,  37.81567 ],
       [126.792623,  37.796244]])

# 제안

In [8]:
n_apartments = len(apartments)
n_parks = len(parks)
n_zones = len(zones)
n_subways = len(subways)
print("n_apartments :",n_apartments)
print("n_parks :",n_parks)
print("n_zones :",n_zones)
print("n_subways :",n_subways)

n_apartments : 216
n_parks : 42
n_zones : 36
n_subways : 9


In [9]:
# euclidean distance
def d(x:np.array, y:np.array) -> float:
    return np.linalg.norm(x-y)
x = np.array([0,3])
y = np.array([4,0])
d(x,y)

5.0

## score 설정

임의의 행복주택 $H$에 대하여, 다음과 같은 사실들을 가정한다.
- 가정1 : $H$에서 공영주차장이 가까이 위치할 수록 $H$의 근처에 쏘카존을 제안할 만하다. (subscript : P)
- 가정2 : $H$에서 쏘카존이 멀리 위치할 수록 $H$의 근처에 쏘카존을 제안할 만하다. (subscript : Z)
- 가정3 : $H$에서 지하철역이 멀리 위치할 수록 $H$의 근처에 쏘카존을 제안할 만하다. (subscript : S)
(지하철이 있으면, 지하철을 타지 쏘카를 타지는 않을 것이다.)
- 가정3 : $H$의 근처에 아파트가 많을수록 $H$의 근처에 쏘카존을 제안할 만하다. (미반영) (subscript : A)

행복주택 $H$에 대하여, 해당 행복주택 근처에 쏘카존을 제안할 만한지를 나타내는 score (score function)를 $f(H)$라고 하자.
<!-- (일반적인 목적함수 e.g. loss function/cost function/error function는 목적함수의 값을 최소화하는 것이 목적이지만, 이번 목적함수는 최대화하는 것이 목적이다.) -->

\begin{align*}
f(H)&=\alpha_P f_P(H) + \alpha_Z f_Z(H) + \alpha_S f_S(H) + \alpha_A f_A(H)\\
f_P(H)&:-(\text{가장 가까운 공영주차장까지의 거리})\\
f_Z(H)&:(\text{가장 가까운 쏘카존까지의 거리})\\
f_S(H)&:(\text{가장 가까운 지하철역까지의 거리})\\
f_A(H)&:(\text{반경 1km 이내의 아파트의 개수})
\end{align*}

단, $\alpha_P$, $\alpha_Z$, $\alpha_S$, $\alpha_A$는 각각의 additive subfunction에 대한 가중치로, 해당 feature들을 어느 정도의 중요도로 반영할지를 나타내는 값이다.
가중치가 모두 1이면, score는 다음과 같은 비교적 간단한 식이 된다.

$$f(H) = -(\text{가장 가까운 공영주차장까지의 거리}) + (\text{가장 가까운 쏘카존까지의 거리}) + (\text{가장 가까운 지하철역까지의 거리}) + (\text{반경 1km 이내의 아파트의 개수})$$

## 계산

In [10]:
# 1 degree of lattitude = 111.1km
# 1 km = 1/111.1 degree of lattitude = 0.009 degree of lattitude
# 각 아파트 A 마다 A와의 거리가 0.009(1km) 이하인 아파트들의 개수를 구한다.

In [11]:
# distances : the distances of the apartment A and every apartment.
# cardinality : the number of apartments which is closer to apartment A than 0.009.
# park_info : the list of cardinalities for every apartment.
apartments_info = []
for happy in happys:
    distances = np.array([d(apartment, happy) for apartment in apartments])
    cardinality = sum(distances <=0.009)
    apartments_info.append(cardinality)
apartments_info = np.array(apartments_info)
apartments_info

array([4, 0, 6, 0, 6, 3, 3, 0])

In [12]:
apartment_names

,아파트 이름,아파트 주소
1,문산주공,경기도 파주시 문산읍 문산로26번길 35
2,동산연립,경기도 파주시 문산읍 독서울5길 5
3,광탄연립,경기도 파주시 광탄면 왕수물길 50
4,동산연립,"경기도 파주시 금정6길 11(금촌동, 동산연립)"
5,동산연립,"경기도 파주시 금정4길 21-6(금촌동, 동산연립)"
...,...,...
212,운정신도시 IPARK,경기도 파주시 심학산로 384
213,휴아림,경기도 파주시 해올로 20
214,별하람마을3단지 서희스타힐스엔에이치에프,"경기도 파주시 가람로 109(와동동, 파주운정3 A4블록)"
215,초롱꽃마을10단지,"경기도 파주시 숲속노을로 60(와동동, 파주운정3 A25블록)"


In [13]:
apartments_info_ = pd.DataFrame({'근처 아파트 개수':apartments_info})
apartments_info_

,근처 아파트 개수
0,4
1,0
2,6
3,0
4,6
5,3
6,3
7,0


In [14]:
# distances : the distances of the apartments and every parking lot
# park_info : the list of (min,index) pair for every apartment.
parks_info = []
for happy in happys:
    distances = [d(happy, park) for park in parks]
    parks_info.append(np.array([np.min(distances),np.argmin(distances)]))
parks_info = np.array(parks_info)
parks_info.shape

(8, 2)

In [15]:
parks_info_ = pd.DataFrame({'주차장 최소거리':parks_info[:,0], 'index':np.round(parks_info[:,1]).astype(int)})
parks_info_['가까운 주차장 주소'] = parks_info_['index'].replace(park_names['주차장 주소'].to_dict())
parks_info_.drop('index', axis=1, inplace=True)
parks_info_

,주차장 최소거리,가까운 주차장 주소
0,0.004603,경기도 파주시 와동동 823-194
1,0.040753,경기도 파주시 야당동 1087
2,0.001472,경기도 파주시 법원리 431-1
3,0.057998,경기도 파주시 맥금동 110-2
4,0.052932,경기도 파주시 맥금동 110-2
5,0.053426,경기도 파주시 야당동 1087
6,0.033588,경기도 파주시 와동동 823-194
7,0.042265,경기도 파주시 야당동 1087


In [16]:
# distances : the distances of the apartments and every zone
# zone_info : the list of (min,index) pair for every apartment.
zones_info = []
for happy in happys:
    distances = [d(happy, zone) for zone in zones]
    zones_info.append(np.array([np.min(distances),np.argmin(distances)]))
zones_info = np.array(zones_info)
zones_info.shape

(8, 2)

In [17]:
zones_info_ = pd.DataFrame({'쏘카존 최소거리':zones_info[:,0], 'index':np.round(zones_info[:,1]).astype(int)})
zones_info_['가까운 존 이름'] = zones_info_['index'].replace(zone_names['존 이름'].to_dict())
zones_info_.drop('index', axis=1, inplace=True)
zones_info_

,쏘카존 최소거리,가까운 존 이름
0,0.004544,가람도서관 앞
1,0.016874,파주중앙공원 앞
2,0.088495,문산어울림아파트 109동
3,0.013635,롯데프리미엄아울렛 파주점
4,0.012213,파주중앙공원 앞
5,0.014929,파주중앙공원 앞
6,0.000641,청암초등학교 옆
7,0.016440,파주중앙공원 앞


In [18]:
# distances : the distances of the apartments and every subway
# subway_info : the list of (min,index) pair for every apartments.
subways_info = []
for happy in happys:
    distances = [d(happy, subway) for subway in subways]
    subways_info.append(np.array([np.min(distances),np.argmin(distances)]))
subways_info = np.array(subways_info)
subways_info.shape

(8, 2)

In [19]:
subways_info_ = pd.DataFrame({'지하철 최소거리':subways_info[:,0], 'index':np.round(subways_info[:,1]).astype(int)})
subways_info_['가까운 역 이름'] = subways_info_['index'].replace(subway_names['역 이름'].to_dict())
subways_info_.drop('index', axis=1, inplace=True)
subways_info_

,지하철 최소거리,가까운 역 이름
0,0.005046,운정
1,0.040403,야당
2,0.090405,문산
3,0.071756,야당
4,0.056593,야당
5,0.053297,야당
6,0.034571,운정
7,0.041931,야당


In [51]:
scores = pd.concat([apartments_info_, parks_info_['주차장 최소거리'], zones_info_['쏘카존 최소거리'],
                    subways_info_['지하철 최소거리']], axis=1)
means = scores.describe().loc['mean'].values
scores['스코어'] = 1/means[0]*(-scores['주차장 최소거리']) + 1/means[1]*scores['쏘카존 최소거리'] + 1/means[2]*scores['지하철 최소거리'] + 1/means[3]*scores['근처 아파트 개수']
scores.insert(0, '행복주택 주소', list(happy_names))
scores.sort_values(by='스코어', ascending=False, inplace=True)
scores


,행복주택 주소,근처 아파트 개수,주차장 최소거리,쏘카존 최소거리,지하철 최소거리,스코어
2,경기도 파주시 법원읍 법원로 12,6,0.001472,0.088495,0.090405,128.603432
4,경기도 파주시 노을빛로 41,6,0.052932,0.012213,0.056593,124.846403
0,경기도 파주시 소리천로 203,4,0.004603,0.004544,0.005046,81.583382
5,경기도 파주시 초롱꽃로 17,3,0.053426,0.014929,0.053297,63.851439
6,경기도 파주시 교하로 20,3,0.033588,0.000641,0.034571,62.567506
3,경기도 파주시 문발로 430,0,0.057998,0.013635,0.071756,3.780506
7,경기도 파주시 숲속노을로 60,0,0.042265,0.016440,0.041931,2.442269
1,경기도 파주시 양지로 112,0,0.040753,0.016874,0.040403,2.382053


In [52]:
save(scores, "results/1d203_0203_scores_파주.png")